# Локальная валидация метрики

In [1]:
# Пример использования подхода из бейзлайна для тестирования модели и 
# расчета метрики через деление hist_data на трейн и валидацию

import gc
import numpy as np
import pandas as pd
from collections import Counter

train = pd.read_csv('train.csv')
val_shown = pd.read_csv('val_shown.csv')
val_hidden = pd.read_csv('val_hidden.csv')

In [2]:
pops = train.groupby('item_id')['pav_order_id'].count().reset_index().sort_values('pav_order_id', ascending=False).head(100)
pops.columns = ['item_id', 'p']
pops['p'] = pops['p'] / train.pav_order_id.nunique()
pops = list(zip(pops.item_id, pops.p))

In [3]:
# из списка кандидатов по совстречаемости удаляем повторяющиеся item_id, сохраняя порядок
def get_unique_recs(recs: list, basket: list, pops: list, top_n: int) -> list:
    rec_dict = {}
    counter = 0

    for k, v in recs:
        if k not in rec_dict and k not in basket:
            rec_dict[k] = v
            counter += 1
        if counter == top_n:
            break

    if counter == top_n:
        return list(rec_dict.keys())
    
    for k, v in pops:
        if k not in rec_dict and k not in basket:
            rec_dict[k] = v
            counter += 1
        if counter == top_n:
            break

    return list(rec_dict.keys())        
    

def rec_by_item(item_id: int, most_freq_dict: dict) -> list:
    return most_freq_dict.get(item_id, None)


# для каждого item_id соберем top_n самых часто встречающихся item_id, отсортируем по частоте и выберем уникальные
def rec_by_basket(basket: list, most_freq_dict: dict, pops: list, top_n: int = 20) -> list:
    
    res = []
    for item in basket:
        recs = rec_by_item(item, most_freq_dict)
        if recs is not None:
            res += recs
    
    res = sorted(res, key=lambda x: x[1], reverse=True)

    return get_unique_recs(res, basket, pops, top_n)

# метрики оцениваются для вектора релевантности. пример:
# реальные item_id, которые приобрел покупатель: [1 ,4, 5, 69]
# рекомендованные алгоритмом item_id: [4, 6, 7, 8, 1, 2, 67, 90]
# тогда вектор релеватности будет выглядеть следующим образом: [1, 0, 0, 0, 1, 0, 0, 0]
# и уже по не му будет расчитываться ndcg
def dcg(
    y_relevance: np.ndarray
) -> float:
    return np.sum([(2**i - 1) / np.log2(k + 1) for (k, i) in enumerate(y_relevance, start=1)])

def ndcg(
    y_relevance: np.ndarray,
    k: int
) -> float:
    if y_relevance.sum() == 0:
        return 0.0
    DCG = dcg(y_relevance[:k])
    IDCG = dcg(-np.sort(-y_relevance)[:k])
    return DCG / IDCG

def apply_relevance(x):
    return [int(item in x['hidden_basket']) for item in x['preds']]

def create_relevance(pred):
    d = pred.copy()
    d['hidden_basket'] = d['hidden_basket'].apply(set)
    d = d.apply(apply_relevance, axis=1)
    return d

def ndcg_full_dataset(d):
    dd = pd.DataFrame(d.to_list()).fillna(0).to_numpy()
    k = dd.shape[1]
    scores = [ndcg(dd[i], k) for i in range(len(dd))]
    return np.mean(scores)

def compute_ndcg_score(pred):
    relevance = create_relevance(pred)
    return ndcg_full_dataset(relevance)

def make_coocurs_dict(train_data):
    tmp = (
        train_data[['item_id', 'pav_order_id']]
        .sort_values(['item_id', 'pav_order_id'])
        .merge(train_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
    )
    tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
    tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))
    tmp2 = train_data.groupby(['item_id'])['pav_order_id'].count().reset_index()
    base = dict(zip(tmp2.item_id, tmp2.pav_order_id))

    most_freq_dict = {k: [(x[0], (x[1]+0.01)/(10+base[k])) for x in v] for (k, v) in tmp1.iteritems()}

    del tmp1, tmp
    gc.collect()
    return most_freq_dict

def create_basket(test_data):
    basket = test_data.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
    return basket

def create_basket_with_hidden(test_data_shown, test_data_hidden):
    basket = test_data_shown.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
    hidden = test_data_hidden.groupby(['pav_order_id'])['item_id'].agg([('hidden_basket', list)])
    basket['hidden_basket'] = hidden['hidden_basket']
    return basket

def make_predictions(test_data_shown, test_data_hidden, most_freq_dict, pops):
    pred = create_basket_with_hidden(test_data_shown, test_data_hidden)
    pred['preds'] = pred['basket'].map(lambda x: rec_by_basket(x, most_freq_dict=most_freq_dict, pops=pops))
    return pred

In [4]:
# соберем словарь встречаемостей - какие item_id покупались чаще с каждым item_id 
most_freq_dict = make_coocurs_dict(train)

In [5]:
# предсказываем
pred = make_predictions(val_shown, val_hidden, most_freq_dict, pops)

In [6]:
# посчитаем скор для всего набора предсказаний # 10 в нормировке, без товаров корзины, вероятности
d_score = compute_ndcg_score(pred)
print(d_score)

0.30814848873154566


# Расчет для теста

In [7]:
hist_data = pd.read_csv('hist_data.csv')

# соберем словарь встречаемостей - какие item_id покупались чаще с 
# каждым item_id 
tmp = (
    hist_data[['item_id', 'pav_order_id']]
    .sort_values(['item_id', 'pav_order_id'])
    .merge(hist_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
)
tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))
tmp2 = hist_data.groupby(['item_id'])['pav_order_id'].count().reset_index()
base = dict(zip(tmp2.item_id, tmp2.pav_order_id))

most_freq_dict_test = {k: [(x[0], (x[1]+0.1)/(10+base[k])) for x in v] for (k, v) in tmp1.iteritems()}

del tmp1, tmp
gc.collect()

0

In [8]:
pops_test = hist_data.groupby('item_id')['pav_order_id'].count().reset_index().sort_values('pav_order_id', ascending=False).head(100)
pops_test.columns = ['item_id', 'p']
pops_test['p'] = pops_test['p'] / hist_data.pav_order_id.nunique()
pops_test = list(zip(pops_test.item_id, pops_test.p))

In [9]:
test = pd.read_csv('test.csv')

pred = test.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
pred['preds'] = pred['basket'].map(lambda x: rec_by_basket(x, most_freq_dict=most_freq_dict_test, pops=pops_test))

In [10]:
for i in range(len(pred)):
    if len(pred.iloc[i]['preds'])!= 20:
        print(i)

In [11]:
pred['preds'].to_csv('pred.csv')